<a href="https://colab.research.google.com/github/DarkEol/AutoML-Ontology/blob/main/AutoSki_DecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#script for building decision tree from meta-features and ontology from decision tree

In [ ]:
!pip install owlready2

In [ ]:
import pandas as pd
import graphviz 
import pickle
from sklearn import tree
from sklearn.tree import export_text
from owlready2 import *

In [ ]:
info = pd.read_csv('featuresInput40.csv')
#print(type(info.columns.values))
#print(info.columns.values[:len(info.columns.values)-1])
#print(info)

#target is last attribute
y = info[info.columns[info.columns.size-1]]
X = info[info.columns.delete(info.columns.size-1)]
print("x len: ", len(X))
#print("X: ", X)
#print("y:", y)

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, y)
#tree.plot_tree(clf)
print("save decision tree into file")
file = open('decisionTree.txt','wb')
pickle.dump(clf, file)
file.close()
print("file saved")

r = export_text(clf, feature_names=list(info.columns.values[:len(info.columns.values)-1]))
print(r)
dot_data = tree.export_graphviz(clf, out_file=None, feature_names=list(info.columns.values[:len(info.columns.values)-1]), class_names=y) 
graph = graphviz.Source(dot_data) 
graph.render("tree") 

In [ ]:
def extendOntology(list_of_nodes):
  #list_of_nodes = ['FewInstances' if x=='NoManyInstances' else x for x in list_of_nodes]
  #list_of_nodes = ['ManyFeatures' if x=='NoFewFeatures' else x for x in list_of_nodes]
  algo = list_of_nodes.pop()
  #print("algo: ", algo)
  info_to_add = onto.AutoSklearnAlgorithm
  for node in list_of_nodes:
    #print(dict_features[node])
    info_to_add = info_to_add & onto.suitableFor.some(dict_features[node])
  #print(info_to_add)
  dict_algos[algo].is_a.append(info_to_add)
  #print(list_of_nodes)

def travelTree(clf, node_id, list_of_nodes):
  left_id = clf.tree_.children_left[node_id]
  right_id = clf.tree_.children_right[node_id]
  feature = info.columns[clf.tree_.feature[node_id]]

  if left_id==-1 and right_id==-1:
    list2 = list_of_nodes.copy()
    #list2.append(y[clf.tree_.value[node_id].argmax()])
    list2.append(clf.classes_[clf.tree_.value[node_id].argmax()])
    extendOntology(list2)
  
  if left_id!=-1:
    list2 = list_of_nodes.copy()
    list2.append("No"+feature)
    travelTree(clf, left_id, list2)
  if right_id!=-1:
    list2 = list_of_nodes.copy()
    list2.append(feature)
    travelTree(clf, right_id, list2)

onto = get_ontology("ClassOntologyEmpty.owl").load()

dict_algos = {
'adaboost' : onto.AdaBoostClassifier,
'gradient_boosting' : onto.GradientBoostingClassifier,
'extra_trees' : onto.ExtraTreesClassifier,
'k_nearest_neighbors' : onto.KNeighborsClassifier,
'lda' : onto.LinearDiscriminantAnalysis,
'liblinear_svc' : onto.LIN_SVC,
'libsvm_svc' : onto.SVM_SVC,
'mlp' : onto.MLPClassifier,
'passive_aggressive' : onto.PassiveAggressiveClassifier,
'random_forest' : onto.RandomForestClassifier}

dict_features = {
    'BinaryClass' : onto.BinaryClass,
    'NoBinaryClass' : onto.NoBinaryClass,
    'StringClass' : onto.StringClass,
    'NoStringClass' : onto.NoStringClass,
    'UnaryAttributes' : onto.UnaryAttributes,
    'NoUnaryAttributes' : onto.NoUnaryAttributes,
    'ManyFeatures' : onto.ManyFeatures,
    'NoManyFeatures' : onto.FewFeatures,
    'FewFeatures' : onto.FewFeatures,
    'NoFewFeatures' : onto.ManyFeatures,
    'ManyInstances' : onto.ManyInstances,
    'FewInstances' : onto.FewInstances,
    'NoManyInstances' : onto.FewInstances,
    'NoFewInstances' : onto.ManyInstances,
    'BinaryAttributes' : onto.BinaryAttributes,
    'NoBinaryAttributes' : onto.NoBinaryAttributes,
    'NumericAttributes' : onto.NumericAttributes,
    'NoNumericAttributes' : onto.NoNumericAttributes,
    'NominalAttributes' : onto.NominalAttributes,
    'NoNominalAttributes' : onto.NoNominalAttributes,
    'StringAttributes' : onto.StringAttributes,
    'NoStringAttributes' : onto.NoStringAttributes,
    'MissingValues' : onto.MissingValues,
    'NoMissingValues' : onto.NoMissingValues,
    'NumericClass' : onto.NumericClass,
    'NoNumericClass' : onto.NoNumericClass
    }

travelTree(clf, 0, list())

onto.save(file = "ontoUpdated.owl")

class_name = 'BinaryClass'
info_to_insert = onto.AutoSklearnAlgorithm & onto.suitableFor.some(onto.class_name)
print(type(info_to_insert))
info_to_insert = info_to_insert & onto.suitableFor.some(onto.ManyFeatures)
print(info_to_insert)

In [ ]:
print("reading classifier from file")
file = open('decisionTree50-mVal.txt','rb')
clfTree = pickle.load(file)
file.close()
print("classifier loaded")
print("type: ", type(clfTree))
r = export_text(clfTree, feature_names=list(info.columns.values[:len(info.columns.values)-1]))
print(r)
dot_data = tree.export_graphviz(clf, out_file=None, feature_names=list(info.columns.values[:len(info.columns.values)-1]), class_names=y) 
graph = graphviz.Source(dot_data) 
graph.render("tree") 